# Check morphology on classification

## Setup
Imports

In [1]:
import glob
import sys
import os
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml
import re
import joblib
from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm
from scipy.ndimage import gaussian_filter
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from collections import defaultdict
import numba as nb
import javabridge
import bioformats
import aicspylibczi as aplc


In [2]:
gc.collect()

28

Move to workdir

In [11]:
# Absolute path
cluster = '/fs/cbsuvlaminck2'

project_workdir = '/workdir/bmg224/manuscripts/mgefish/code/fig_5/2023_02_09_hiprmega/HiPR_MeGA'

os.chdir(cluster + project_workdir)
os.getcwd()  # Make sure you're in the right directory

'/fs/cbsuvlaminck2/workdir/bmg224/manuscripts/mgefish/code/fig_5/2023_02_09_hiprmega/HiPR_MeGA'

Load config file

In [12]:
config_fn = 'config.yaml' # relative path to config file from workdir

with open(config_fn, 'r') as f:
    config = yaml.safe_load(f)

with open(config['config_mega'], 'r') as f:
    config_mega = yaml.safe_load(f)


with open(config['config_hipr'], 'r') as f:
    config_hipr = yaml.safe_load(f)


Special imports 

In [13]:
%reload_ext autoreload
%autoreload 2
sys.path.append(config_mega['pipeline_path'] + '/' + config_mega['functions_path'])
sys.path
import image_plots as ip
import fn_hiprfish_classifier as fhc
import fn_spectral_images as fsi


Sample names

In [14]:
input_table_mge = pd.read_csv('../MeGAFISH/' + config_mega['input_table_fn'])
sample_names_mge = input_table_mge.sample_name.values
sample_names_mge

array(['2023_02_09_hiprmega_probe_capsb_fov_02',
       '2023_02_09_hiprmega_probe_capsb_fov_03',
       '2023_02_09_hiprmega_probe_capsb_fov_04',
       '2023_02_09_hiprmega_probe_ec_fov_01',
       '2023_02_09_hiprmega_probe_laur_fov_01',
       '2023_02_09_hiprmega_probe_laur_fov_02',
       '2023_02_09_hiprmega_probe_strepr_fov_01',
       '2023_02_09_hiprmega_probe_strepr_fov_02',
       '2023_02_09_hiprmega_probe_terml_fov_01',
       '2023_02_09_hiprmega_probe_terml_fov_02',
       '2023_02_09_hiprmega_probe_terml_fov_03'], dtype=object)

In [15]:
input_table_hipr = pd.read_csv('../HiPRFISH/' + config_hipr['images']['image_list_table'])
sample_names_hipr = input_table_hipr.IMAGES.values
sample_names_hipr


array(['2023_02_09_hiprmega_probe_capsb_fov_01',
       '2023_02_09_hiprmega_probe_capsb_fov_02',
       '2023_02_09_hiprmega_probe_capsb_fov_03',
       '2023_02_09_hiprmega_probe_capsb_fov_04',
       '2023_02_09_hiprmega_probe_ec_fov_01',
       '2023_02_09_hiprmega_probe_ec_fov_02',
       '2023_02_09_hiprmega_probe_laur_fov_01',
       '2023_02_09_hiprmega_probe_laur_fov_02',
       '2023_02_09_hiprmega_probe_laur_fov_03',
       '2023_02_09_hiprmega_probe_strepr_fov_01',
       '2023_02_09_hiprmega_probe_strepr_fov_02',
       '2023_02_09_hiprmega_probe_terml_fov_01',
       '2023_02_09_hiprmega_probe_terml_fov_02',
       '2023_02_09_hiprmega_probe_terml_fov_03'], dtype=object)

## Look at the classif
Load the classif files

In [24]:
nlas=3
probe_design = pd.DataFrame({'laser_present':[100,11,11,110,110]})
present_col = probe_design['laser_present'].values
present_str = [str(lp).zfill(nlas) for lp in present_col]
present_arr = np.array([[int(l) for l in lp] for lp in present_str])
present_arr.T


array([[1, 0, 0, 1, 1],
       [0, 1, 1, 1, 1],
       [0, 1, 1, 0, 0]])